# XGBoost timeseries price prediction

this is a test to see how best to train it to predict the next day only.

## dataset

In [105]:
import time 
import json
import requests
import datetime as dt
import pandas as pd
from satori import config 
from satori.lib.apis import disk

In [106]:
streamId = 'simpleEURCleaned'
df = pd.read_csv(config.root('lib', 'spoof', f'{streamId}.csv'))

In [107]:
df['Next Day'] = df['Close'].shift(-1)
df = df.dropna()
df = df.reset_index(drop=True)
past = df.iloc[:100]
future = df.iloc[100:]
features = ['High', 'Low', 'Close']
target = 'Next Day'

In [108]:
df

,High,Low,Close,Next Day
0,0.837240,0.830560,0.835770,0.827200
1,0.837100,0.825830,0.827200,0.824880
2,0.828020,0.824400,0.824880,0.827750
3,0.830290,0.823450,0.827750,0.820550
4,0.828780,0.820280,0.820550,0.818330
...,...,...,...,...
4588,0.847230,0.843597,0.844340,0.846088
4589,0.847020,0.844500,0.846088,0.845600
4590,0.846070,0.843668,0.845600,0.846600
4591,0.849574,0.846200,0.846600,0.846600


In [109]:
past

,High,Low,Close,Next Day
0,0.83724,0.83056,0.83577,0.82720
1,0.83710,0.82583,0.82720,0.82488
2,0.82802,0.82440,0.82488,0.82775
3,0.83029,0.82345,0.82775,0.82055
4,0.82878,0.82028,0.82055,0.81833
...,...,...,...,...
95,0.82932,0.82672,0.82829,0.83766
96,0.83963,0.82761,0.83766,0.83703
97,0.84196,0.83521,0.83703,0.83424
98,0.84034,0.83403,0.83424,0.83313


In [110]:
future

,High,Low,Close,Next Day
100,0.833130,0.827950,0.832850,0.844020
101,0.845450,0.831950,0.844020,0.846740
102,0.845950,0.839490,0.846740,0.838570
103,0.848100,0.838150,0.838570,0.843880
104,0.847310,0.837590,0.843880,0.843030
...,...,...,...,...
4588,0.847230,0.843597,0.844340,0.846088
4589,0.847020,0.844500,0.846088,0.845600
4590,0.846070,0.843668,0.845600,0.846600
4591,0.849574,0.846200,0.846600,0.846600


## imports

In [111]:
#!pip install --upgrade pip --user
#!pip install xgboost
#!pip install seaborn

In [112]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
big = {'figsize':(16,10)}
verbose = False

## train test split

In [123]:
def getSplit(i):
    return train_test_split(
        df.loc[:2299+i, features], 
        df.loc[:2299+i, target], 
        test_size=.2, # 2 
        shuffle=False)
X_train, X_test, y_train, y_test = getSplit(0)

In [124]:
X_train.tail(2)

,High,Low,Close
1838,0.74729,0.73873,0.74749
1839,0.74610,0.73975,0.74338


In [125]:
X_test

,High,Low,Close
1840,0.743440,0.735890,0.74250
1841,0.738170,0.730990,0.73424
1842,0.736600,0.730780,0.73292
1843,0.732590,0.728990,0.73067
1844,0.732980,0.726940,0.72939
...,...,...,...
2295,0.768280,0.764584,0.76516
2296,0.767900,0.764500,0.76790
2297,0.771992,0.765000,0.76510
2298,0.773772,0.769290,0.77080


In [126]:
y_train

0       0.82720
1       0.82488
2       0.82775
3       0.82055
4       0.81833
         ...   
1835    0.74737
1836    0.75279
1837    0.74749
1838    0.74338
1839    0.74250
Name: Next Day, Length: 1840, dtype: float64

In [127]:
y_test

1840    0.73424
1841    0.73292
1842    0.73067
1843    0.72939
1844    0.72870
         ...   
2295    0.76790
2296    0.76510
2297    0.77080
2298    0.77130
2299    0.77290
Name: Next Day, Length: 460, dtype: float64

## fit

In [128]:
def getScore(X_train, X_test, y_train, y_test):
    reg = XGBRegressor(n_estimators=500, learning_rate=0.01)
    reg.fit(X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            #eval_metric='mae', 
            #early_stopping_rounds=10,
            verbose=False)
    return reg.score(X_test, y_test)

## test full train version

In [122]:
scores = []
for i in range(2296):
    scores.append(getScore(*getSplit(i)))
score = sum(scores) / len(scores)
print(score)

-1183.1968869482153


## test .8 split train version

In [129]:
scores = []
for i in range(2296):
    scores.append(getScore(*getSplit(i)))
score = sum(scores) / len(scores)
print(score)

0.6594544981366467
